<a href="https://colab.research.google.com/github/napoles-uach/DGL_Smell/blob/main/dgl_smell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install dgl

In [ ]:
pip install ogb

In [ ]:
pip install rdkit-pypi

In [ ]:
!wget https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv

In [55]:
# modificación para moleculas
import dgl
from dgl.data import DGLDataset
import torch
import torch as th
import os
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict) 
from rdkit import Chem
import numpy as np

In [56]:
def smiles2graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)
    #mol = Chem.MolFromSmiles('CN1C=NC2=C1C(=O)N(C(=O)N2C)C')
    A = Chem.GetAdjacencyMatrix(mol)
    A = np.asmatrix(A)
    nnodes=len(A)
    nz = np.nonzero(A)
    edge_list=[]
    src=[]
    dst=[]



    for i in range(nz[0].shape[0]):
      #lista=(nz[0][i],nz[1][i])
      #edge_list.append(lista)
      src.append(nz[0][i])
      dst.append(nz[1][i])

    #g = dgl.graph(edge_list[0],edge_list[1])
    # add 34 nodes into the graph; nodes are labeled from 0~33
    #g.add_nodes(nnodes)
    # all 78 edges as a list of tuples
    #u, v = th.tensor(src), th.tensor(dst)
    u, v = src, dst
    g = dgl.graph((u, v))
    bg=dgl.to_bidirected(g)
 
    # add edges two lists of nodes: src and dst
    #src, dst = tuple(zip(*edge_list))
    #g = dgl.graph(src,dst)
    #g.add_edges(src, dst)
    
    return bg

#class MolDataset(DGLDataset):
#    def __init__(self):
#        super().__init__(name='mol_club')

#    def process(self):

#        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
#        self.graph.ndata['feat'] = node_features
#        self.graph.ndata['label'] = node_labels
#        self.graph.edata['weight'] = edge_features



#    def __getitem__(self, i):
#        return self.graph

#    def __len__(self):
#        return 1

In [57]:
###############
def feat_vec(smiles_string):
    # atoms
    mol = Chem.MolFromSmiles(smiles_string)
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)
    return x


In [58]:
import pandas as pd
df=pd.read_csv('train.csv')

In [59]:
lista_mols=df['SMILES'].to_list()

In [60]:
grafo1=smiles2graph(lista_mols[69])

In [61]:
lista_mols[69]

'CCC=CC=CCO'

In [62]:
grafo1.ndata['feat']=torch.tensor(feat_vec(lista_mols[69])) 

In [63]:
grafo1.ndata#['label']=torch.tensor(onehot[69])

{'feat': tensor([[5, 0, 4, 5, 3, 0, 2, 0, 0],
        [5, 0, 4, 5, 2, 0, 2, 0, 0],
        [5, 0, 3, 5, 1, 0, 1, 0, 0],
        [5, 0, 3, 5, 1, 0, 1, 0, 0],
        [5, 0, 3, 5, 1, 0, 1, 0, 0],
        [5, 0, 3, 5, 1, 0, 1, 0, 0],
        [5, 0, 4, 5, 2, 0, 2, 0, 0],
        [7, 0, 2, 5, 1, 0, 2, 0, 0]])}

In [64]:
torch.tensor(onehot[69])

tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.], dtype=torch.float64)

In [65]:
lista_senten=df['SENTENCE'].to_list()
olores=[]
for olor in lista_senten:
  olor=olor.split(",")
  olores.append(olor)
# da formato correcto a las sentencias en forma de lista
from pandas.core.common import flatten
olores=list(flatten(olores))
#crea una sola lista con los olores
olores = list(dict.fromkeys(olores))
#elimina repeticiones
olores.sort()
#ordena alfabeticamente
inoh=[]
for olor in lista_senten:
  olor=olor.split(",")
  indices=[]
  for t in olor:
    if t in olores:
      indices.append(olores.index(t))
  inoh.append(indices)

onehot=[]
for i in inoh:
  ohv=np.zeros(len(olores))
  ohv[i]=1
  onehot.append(ohv)

In [66]:
j=0
ind=[]
source=[]
dest=[]
nodos=[]
feats=[]
graphs=[]
execptions=[]
for mol in lista_mols:
  
  g_mol=smiles2graph(mol)
  #print(g_mol)
  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol)) 
  #print(x)
  except:
    execptions.append(j)

  #g_mol.ndata['label']=torch.tensor(onehot[j])
   
  j+=1
  graphs.append(g_mol) #agrega grafo de mol
  # features
  #feats.append(feat_vec(mol))

  #nodos.append(len(g_mol.nodes()))
  #src,dst=g_mol.edges()
  #src=src.tolist()
  #dst=dst.tolist()
  #for i in range(len(src)):
  #  ind.append(j-1)
  #  source.append(src[i])
  #  dest.append(dst[i])


In [67]:
execptions

[70,
 265,
 305,
 454,
 1542,
 1574,
 1915,
 1995,
 2087,
 2149,
 2389,
 2540,
 2587,
 2899,
 3347,
 3671,
 4008,
 4014,
 4250]

In [70]:
for i in execptions:
  graphs.pop(i)
  labels.pop(i)

In [69]:
labels=onehot

In [71]:
class SyntheticDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        #edges = pd.read_csv('./graph_edges.csv')
        #properties = pd.read_csv('./graph_properties.csv')
        self.graphs = graphs
        self.labels = torch.LongTensor(labels)

        

        # Convert the label list to tensor for saving.
        #self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = SyntheticDataset()
graph, label = dataset[0]
print(graph, label)

Graph(num_nodes=14, num_edges=28,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
